<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Algorithme de Needleman et Wunsch

## Version récursive

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Les coûts

Commençons par nous donner les coûts d'insertion et de substitution, tels que décrits dans la vidéo de la séquence 7. Pour cela nous allons supposer l'existence de deux fonctions&nbsp;:
 * `insertion_cost(base)`
 * `substitution_cost(base1, base2)`

L'algorithme utilise ces fonctions, qu'il sera possible de modifier séparément sans avoir à retoucher à l'algorithme principal.

### L'algorithme récursif

On veut écrire une fonction `needleman_wunsch_rec`. Pour éviter de créer des sous-chaines à chaque appel récursif, on va passer à la fonction à la fois les chaines `adn1` et `adn2`, et les indices `i` et `j`.

##### Condition d'arrêt

L'algorithme récursif consiste donc à rappeler la fonction elle-même avec des indices plus petits. La récursivité **s'arrête pour nous sur les bords supérieur et gauche du cadre**. C'est à dire que si `i` ou `j` vaut `0`, on calcule un coût qui est la somme des coûts d'insertion de la chaine qui n'est pas vide.

C'est-à-dire que si par exemple on a&nbsp;:

In [ ]:
adn1 = "ACGT"
adn2 = "GTAC"

Alors pour un point du bord supérieur&nbsp;: `needleman_wunsch_rec(adn1, adn2, 2, 0)` doit retourner 
  `insertion_cost('A') + insertion_cost('C')` 
  
et de la même façon sur le bord gauche&nbsp;: si on reçoit en argument `i=0` et `j=2` alors on doit calculer `insertion_cost('G') + insertion_cost('T')`

##### L'algorithme

Un dernier point à remarquer avant de nous lancer, concerne les poteaux. À nouveau les indices en python commencent à 0, aussi nous devons corriger un peu les formules qui sont données dans la vidéo. Lorsque nous avons en entrée par exemple les chaines "ABC" et "AC", nous devons calculer en fait 4x3 valeurs pour remplir le tableau, comme on le voit sur cette figure&nbsp;:

![Indices pour le calcule](media/nw-indices.png)

C'est pourquoi dans notre cas la formule principale est 

    cost (i, j) = min ( cost(i-1, j-1) + substitution(adn1[i-1], adn2[j-1]),
                        ...)
                        
et non pas `substitution(adn1[i], adn2[j])`

Voici donc le code de cet algorithme, dans sa version récursive&nbsp;:

In [ ]:
def needleman_wunsch_rec(adn1, adn2, i=None, j=None):
    """
    Calcule l'écart entre deux chaines d'ADN selon
    l'algorithme de Needleman et Wunsch
    en version récursive
    
    Utilise les fonctions
     * insertion_cost(base) et
     * substitution_cost(base1, base2)
     
    L'appelant en général ne spécifie pas i et j, qui sont utilisés
    lors des appels récursifs
    """
    # si on n'a pas précisé i ou j cela signifie 
    # qu'on veut travailler sur toute la chaine
    i = i if i is not None else len(adn1)
    j = j if j is not None else len(adn2)

    ### la condition d'arrêt
    # le bord supérieur
    if j == 0:
        return sum(insertion_cost(base) for base in adn1[:i])
    # le bord gauche
    elif i == 0:
        return sum(insertion_cost(base) for base in adn2[:j])
        
    # dans le cas général on peut appliquer la formule telle quelle
    return min(
        # substitution
        needleman_wunsch_rec(adn1, adn2, i-1, j-1) + substitution_cost(adn1[i-1], adn2[j-1]),
        # insertion
        needleman_wunsch_rec(adn1, adn2, i, j-1) + insertion_cost(adn2[j-1]),
        # insertion
        needleman_wunsch_rec(adn1, adn2, i-1, j) + insertion_cost(adn1[i-1]))

### Une version simple des coûts

Avant de pouvoir utiliser cette fonction il nous faut donc fournir une implémentation pour les deux fonctions de coût; je vous en donne la version la plus simple, où les coûts d'insertion sont forfaitairement de `1`, et les substitutions sont aussi forfaitairement à `1` si on change de lettre, et `0` sinon&nbsp;:

In [ ]:
# la fonction d'insertion la plus simple possible
def insertion_cost(base):
    return 1

In [ ]:
# la fonction de substitution la plus simple possible
def substitution_cost(base1, base2):
    return 1 if base1 != base2 else 0

In [ ]:
# exemples
print("insertion", insertion_cost('A'))
# c'est gratuit 
print("remplacement à l'identique", substitution_cost('A', 'A'))
# ou forfaitairement 1
print("remplacement différent", substitution_cost('A', 'T'))

### L'algorithme en (in)action

Commençons sur un tout petit exemple, avec deux brins petits et presque identiques&nbsp;:

In [ ]:
needleman_wunsch_rec("ACTG", "ACTC")

ce qui en l'occurrence correspond aussi à la distance de Hamming. Pour apprécier la puissance de cet algorithme voyons le résultat sur un brin dans lequel on a inséré une base vers le début&nbsp;:

In [ ]:
needleman_wunsch_rec("ACGTAGC", 
                     "ACTGTAGC")
#                       ^           

Par contre cette version de l'algorithme est **très inefficace**. On peut pousser un tout petit peu sur la longueur des chaines, mais voyez le temps de calcul avec une chaine de longueur 10, vous observez que le temps de réaction commence à se faire sentir&nbsp;:

In [ ]:
needleman_wunsch_rec("ACTGCCAAC", "ACTGCGCAAC")

### Analyse de complexité

Vous l'avez sans doute déjà compris à ce stade, mais cet algorithme est inutilisable en l'état, car il est un peu stupide, en ce sens que l'on recalcule sans cesse les mêmes valeurs. Le résultat est que sa complexité est **exponentielle**. Pour bien le voir je vous propose une récriture simplifiée de l'algorithme, mains instrumentée pour avec une impression pour bien voir ce qui se passe&nbsp;:

In [ ]:
counter = 0

# une version bavarde mais similaire
def nw(adn1, adn2, i, j):
    global counter
    counter += 1
    print("le couple ({}, {})".format(i, j))
    if i == 0 or j == 0:
        return (i+j)
    return min(
        # en diagonale
        nw(adn1, adn2, i-1, j-1) + (adn1[i] != adn2[j]),
        # a gauche
        nw(adn1, adn2, i, j-1) + 1,
        # en montant
        nw(adn1, adn2, i-1, j) + 1)

In [ ]:
counter = 0
nw("ACGT", "ACGT", 2, 2)
print("on a appelé nw {} fois".format(counter))

Vous pouvez voir que l'on appelle **trois fois** la fonction avec `(0, 1)`. Si on passe juste à l'entier suivant, on passe de 19 appels à .. 94 !

In [ ]:
counter = 0
nw("ACGT", "ACGT", 3, 3)
print("on a appelé nw {} fois".format(counter))

C'est cette complexité exponentielle qui rend l'algorithme récursif prohibitif, et qui rend nécessaire une version itérative, dont la complexité va être plus raisonnable.